In [11]:
import numpy as np
import cv2 as cv
import matplotlib as plt
import glob
import imutils
import os
import math

In [3]:
# gradient operator
def Gradient(image,sobel):
    width = image.shape[0]
    height= image.shape[1]
    
    #intialize the INTESITY matrix 
    intensity = np.zeros((width,height),np.float32)
     
    for r in range(width):
        for c in range(height):
            if r>=1 or r<=width-2 and c>=1 or c<=height-2:
                for i in range(3):
                    for j in range(3):
                        intensity[r][c]= intensity[r][c] + sobel[i][j]*image[r-i-1][c-j-1]
            else :
                intensity[r][c] = image[r][c]
                
                
    return intensity

In [4]:
#HarrisEdge and Corner Detection algorithm

def HarrisCornerDetection(img):
    
    #Convolution mask in horizontal and vertical direction using sobel operator 
    
    sobel_X = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    sobel_Y =np.array([[-1,-2,-1],[0, 0, 0],[1,2,1]])
    
    w = img.shape[0]
    h = img.shape[1]
    
    # Gradient of image using Sobel operator
    ImgX = Gradient(img, sobel_X)
    ImgY = Gradient(img, sobel_Y)
    
    
    
    for i in range(w):
        for j in range(h):
            if ImgY[i][j] < 0:
                ImgY[i][j] *= -1
            
            if ImgX[i][j] < 0:
                ImgX[i][j] *= -1
                
    ImgXX = np.square(ImgX)
    ImgYY = np.square(ImgY)
    
    ImgXY = np.multiply(ImgX, ImgY)
    
    Sigma = 5
    ImgXX = cv.GaussianBlur(ImgXX, (3,3), Sigma)
    ImgYY = cv.GaussianBlur(ImgYY, (3,3), Sigma)
    ImgXY = cv.GaussianBlur(ImgXY, (3,3), Sigma)
    
    k = 0.04
    R = np.zeros((w, h), np.float32)
    # For every pixel find the corner strength
    for row in range(w):
        for col in range(h):
            M = np.array([[ImgXX[row][col], ImgXY[row][col]], [ImgXY[row][col], ImgYY[row][col]]])
#             l1,l2 = np.linalg.eigvals(M_bar)
#             print(l1)
#             print(l2)
            # Determinant and scaled trace of matrix 
            R[row][col] = np.linalg.det(M) - (k * np.square(np.trace(M)))
    return R




In [5]:
def load_image(img_path):

    img = cv.imread(img_path, cv.IMREAD_COLOR)

    h, w, _ = img.shape
    #print(h,w)
    if h > 800 or w > 600:
        print("resizing image to 800 x 500")
        img = cv.resize(img, (500, 800))

    return img

In [6]:
def Harris_main(img):
   
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    bgr = cv.cvtColor(gray, cv.COLOR_GRAY2RGB)

    w = gray.shape[0]
    h = gray.shape[1]
    R = HarrisCornerDetection(gray)


# This parameter will need tuning based on the use-case
    CornerStrengthThreshold = 600000

# Plot detected corners on image
    radius = 2
    color = (0, 0, 255)  # Green
    thickness = 1

    cornerList = []
    #comapring If the R value is greater than the threshold or not
    for row in range(w):
        for col in range(h):
            if R[row][col] > CornerStrengthThreshold:
            # print(R[row][col])
                max = R[row][col]

            # Local non-maxima suppression
                skip = False
                for nrow in range(5):
                    for ncol in range(5):
                        if row + nrow - 2 < w and col + ncol - 2 < h:
                            if R[row + nrow - 2][col + ncol - 2] > max:
                                skip = True
                                break

                if not skip:
                # Point is expressed in x, y which is col, row
               # cv.circle(bgr, (col, row), radius, color, thickness)
                    cv.rectangle(img, (col-radius, row-radius),(col+radius,row+radius), color, thickness)
                    cornerList.append((row, col))
                    
    return (img,cornerList,R)

In [11]:
input_path = 'D:\Mtech 2022-24\Semester 2\Computer Vision Assignment\Assignment2\input\input7.jpg'

img = load_image(input_path)
img,cornerList,R = Harris_main(img)


cv.imshow("Corners_Detection", img)
outname = 'D:\Mtech 2022-24\Semester 2\Computer Vision Assignment\Assignment2\input\output\input7_out.png' + ".png"
cv.imwrite(outname, img)

cv.waitKey(0)
cv.destroyAllWindows()


resizing image to 800 x 500


## SSD CALCULATION

In [7]:
def calculate_ssd(img1,img2,cornerList1,cornerList2,R1,R2):
    l1= len(cornerList1)
    l2= len(cornerList2)
    print(l1,l2)
#     R1 = HarrisCornerDetection(img1)
#     R2 = HarrisCornerDetection(img2)
    corner_map=[]
    for i in range (l1):
        x,y= cornerList1[i]
        #print(cornerList1[i])
        coord = [x,x,y,y]
        
        if (x-10>=0):
            coord[0]=x-10
        else:
            coord[0]=0
        if(x+11<img2.shape[0]):
            coord[1]=x+11
        else:
            coord[1]=img2.shape[0]
        
        if (y-10>=0):
            coord[2]=y-10
        else:
            coord[2]=0
        if(y+11<img2.shape[1]):
            coord[3]=y+11
        else:
            coord[3]=img2.shape[1]
        
        #print(coord)
        min_ssd = np.inf
        best_match = None
        for j in range (l2):
            #print(x,y,min_ssd)
            if ((cornerList2[j][0]>=coord[0] and cornerList2[j][0]<coord[1] and cornerList2[j][1]>=coord[2] and cornerList2[j][1]<coord[3] )==True):
                ssd = (R1[x][y]-R2[cornerList2[j][0]][cornerList2[j][1]])**2
                if ssd < min_ssd :
                    min_ssd = ssd
                    best_match = cornerList2[j]
        #print(best_match,min_ssd)
        corner_map.append([cornerList1[i],best_match,min_ssd])
            
    return corner_map


In [13]:
input_path1 = 'D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\input\\input7.jpg'
#input_path1 = 'D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\New Dataset\\1\\image 0.jpg'


img1 = load_image(input_path1)
_,cornerList1,R1 = Harris_main(img1)

resizing image to 800 x 500


In [12]:
#input_path2 = 'D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\New Dataset\\1\\image 1.jpg'
input_path2 = 'D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\input\\input7.jpg'

img2 = load_image(input_path2)
img2,cornerList2,R2 = Harris_main(img2)

resizing image to 800 x 500


In [16]:

# Convert frames to grayscale
frame1_gray = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
frame2_gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

mapl =calculate_ssd(frame1_gray,frame2_gray,cornerList1,cornerList2,R1,R2)

132 132


In [ ]:
len(mapl)

In [17]:
new_image = cv.hconcat([img1, img2])
color = (0, 0, 255)  # Green
thickness = 1
k =len(mapl)
maxi = 0
for i in range (k):
    if(mapl[i][2]>maxi):
        maxi = mapl[i][2]

threshold = (0.01)*maxi
for i in range (0,k):
    if(mapl[i][2]== np.Inf):
        x=0
    elif(mapl[i][2]<=threshold):
        cv.line(new_image,(mapl[i][0][1],mapl[i][0][0]),(img1.shape[1]+ mapl[i][1][1],mapl[i][1][0]),color,thickness)

outname = 'D:\Mtech 2022-24\Semester 2\Computer Vision Assignment\Assignment2\input\output\input4_SSD_out.png' + ".png"
cv.imwrite(outname, new_image)

cv.imshow("images",new_image)
cv.waitKey(0)
cv.destroyAllWindows()


## Affine Matrix 

In [8]:
#path = 'D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\New Dataset\\2\\*.jpg'
path = 'D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\New Dataset1\\1\\*.jpg'

images =[]
image = []
# Loop through all the image files in the folder
for file in glob.glob(path):
    
    # Read the image file
    x = load_image(file)
    images.append(load_image(file))
    image.append(load_image(file))


resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 800 x 500
resizing image to 80

In [20]:
cv.imshow("hh",image[0])
cv.waitKey(0)
cv.destroyAllWindows()


In [9]:
cornerList = []
R = []
for i in range (len(images)):
   # image= images[i]
    print("Calculating Corner Points for %d image :" %(i+1))
    _,cornerList1,R1 = Harris_main(images[i])
    cornerList.append(cornerList1)
   # image.append(image1)
    R.append(R1)
    print("done for %d : " %(i+1))

Calculating Corner Points for 1 image :
done for 1 : 
Calculating Corner Points for 2 image :
done for 2 : 
Calculating Corner Points for 3 image :
done for 3 : 
Calculating Corner Points for 4 image :
done for 4 : 
Calculating Corner Points for 5 image :
done for 5 : 
Calculating Corner Points for 6 image :
done for 6 : 
Calculating Corner Points for 7 image :
done for 7 : 
Calculating Corner Points for 8 image :
done for 8 : 
Calculating Corner Points for 9 image :
done for 9 : 
Calculating Corner Points for 10 image :
done for 10 : 
Calculating Corner Points for 11 image :
done for 11 : 
Calculating Corner Points for 12 image :
done for 12 : 
Calculating Corner Points for 13 image :
done for 13 : 
Calculating Corner Points for 14 image :
done for 14 : 
Calculating Corner Points for 15 image :
done for 15 : 


In [12]:
def correspondence_pts(mapl):
    k =len(mapl)
    mapl2 =[]
    maxi = 0
    for i in range (k):
        if(mapl[i][2]>maxi):
            maxi = mapl[i][2]

    threshold = (0.01)*maxi
    for i in range (0,k):
        if(mapl[i][2]== np.Inf):
            x=0
        elif(mapl[i][2]<=threshold):
            mapl2.append(mapl[i])
            
    return mapl2

In [13]:
mapList = []
for j in range (len(images)-1):
    mapl =calculate_ssd(cv.cvtColor(images[j], cv.COLOR_BGR2GRAY),cv.cvtColor(images[j+1], cv.COLOR_BGR2GRAY),cornerList[j],cornerList[j+1],R[j],R[j+1])
    mapl = correspondence_pts(mapl)
    mapList.append(mapl)

2118 2087
2087 2097
2097 2079
2079 2104
2104 2111
2111 2103
2103 2124
2124 2075
2075 2093
2093 2111
2111 2135
2135 2079
2079 2099
2099 2066


In [ ]:
len(mapList)

In [14]:
affines = []
for a in range (len(mapList)):
    temp =mapList[0]
    matching = np.zeros((2*len(temp),6),dtype=int)
    for i in range (0,2*len(temp),1):
        if(i%2 == 0):
            x=int(i/2)
            matching[i][0]=temp[x][0][1]
            matching[i][1]=temp[x][0][0]
            matching[i][2]=1
            matching[i][3]=0
            matching[i][4]=0
            matching[i][4]=0
        elif(i%2==1):
            x=int(i/2)
            matching[i][0]=0
            matching[i][1]=0
            matching[i][2]=0
            matching[i][3]=temp[x][0][1]
            matching[i][4]=temp[x][0][0]
            matching[i][5]=1
    
    variable = np.zeros((2*len(temp),1),dtype=int)
    for i in range(2*len(temp)):
        if(i%2 == 0):
            x=int(i/2)
            variable[i]=temp[x][1][1]
        elif(i%2==1):
            x=int(i/2)
            variable[i]=temp[x][1][0]
    straightAffine = np.linalg.lstsq(matching,variable,rcond=-1)[0]
#         print(straightAffine)
    shape = (3,3)
    affineMatrix = np.zeros(shape)
    affineMatrix[0][0]= straightAffine[0]
    affineMatrix[0][1] = straightAffine[1]
    affineMatrix[0][2] = straightAffine[2]
    affineMatrix[1][0] = straightAffine[3]
    affineMatrix[1][1] = straightAffine[4]
    affineMatrix[1][2] = straightAffine[5]
    affineMatrix[2][2]=1
    affines.append(affineMatrix)

In [15]:
len(affines)

14

In [16]:
affines[1]

array([[ 1.00086920e+00, -1.86029077e-04, -8.28708802e-01],
       [-5.28027749e-04,  9.99096422e-01,  6.72120954e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [17]:
affine_cumm = []
affine_cumm.append(affines[0])
for i in range(1,len(affines)):
    affine_cumm.append(np.matmul(affines[i],affine_cumm[i-1]))
    
M=affine_cumm[len(affines)-1]
M

array([[ 1.01224670e+00, -2.60388495e-03, -1.16791063e+01],
       [-7.39090646e-03,  9.87432849e-01,  9.39446585e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [18]:
len(affine_cumm)

14

In [19]:
r = image[0].shape[0]
c = image[0].shape[1]
ty = int(np.ceil(np.abs(M[1][2])))
tx = int( np.ceil(np.abs(M[0][2])))
e=6
rows = r + ty + 6
cols = c +tx + 6
final_image = np.zeros((rows, cols, 3))
final_image.shape


(816, 518, 3)

In [20]:

for i in range(r):
    for j in range(c):
        final_image[i][j] = image[0][i][j]
        
for k in range(len(image)-1): #len(image)
    for i in range(rows):
        for j in range(cols):
            shape = (3,1)
            point = np.zeros(shape)
            new_point = np.zeros(shape)
            point[0][0] = j
            point[1][0] = i
            point[2][0] = 1
            new_point = np.matmul(affine_cumm[k], point)
            x_prime = new_point[0][0]
            y_prime = new_point[1][0]
            if(x_prime<0 or y_prime<0 or x_prime >= c or y_prime >= r):
                continue
                
            x_ceil = math.ceil(x_prime)
            if(x_ceil >= c ):
                x_ceil = c - 1

            y_ceil = math.ceil(y_prime)
            if(y_ceil >= r):
                y_ceil = r  -1 

            x_floor = math.floor(x_prime)
            if(x_floor >= c ):
                x_floor = c - 1

            y_floor = math.floor(y_prime)
            if(y_floor >= r):
                y_floor = r-1 
            
            sum_intensity = np.zeros(3)
            int1 = abs(y_floor - i) * abs(x_floor - j)
            int2 = abs(y_floor - i) * abs(x_ceil - j)
            int3 = abs(y_ceil - i) * abs(x_floor - j)
            int4 = abs(y_ceil - i) * abs(x_ceil - j)
            totalIntensity = int1 + int2 + int3 + int4
            if (totalIntensity == 0):
                final_image[i][j] = image[k+1][y_floor][x_floor]
            else :
                sum_intensity = (int1/totalIntensity) * image[k+1][y_floor][x_floor] + (int2/totalIntensity) * image[k+1][y_floor][x_ceil]  + (int3/totalIntensity) * image[k+1][y_ceil][x_floor]  + (int4/totalIntensity) * image[k+1][y_ceil][x_ceil]
                final_image[i][j] = sum_intensity

In [ ]:
# for i in range(r):
#     for j in range(c):
#         final_image[i][j] = image[0][i][j]
# #cv.imshow("image5",final_image)
# cv.imwrite('D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\New Dataset\\pano\\check.jpg',final_image)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [21]:
#cv.imshow("image5",final_image)
cv.imwrite('D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\New Dataset\\pano\\Panoroma_1.jpg',final_image)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

def stitch_images(img1, img2, affine_matrix):
    # Find the dimensions of the output image
    height = img1.shape[0]
    width = img1.shape[1]+1
    corners = np.array([[0, 0], [0, height], [width, 0], [width, height]])
    transformed_corners = cv2.transform(np.float32([corners]), affine_matrix)[0]
    panorama_width = int(max(transformed_corners[:, 0])) + 1
    panorama_height = int(max(transformed_corners[:, 1])) + 1

    # Create an output image and stitch the input images using bilinear interpolation
    panorama = np.zeros((panorama_height, panorama_width, 3), np.uint8)
    panorama[:height, :width] = img1
    for x in range(width):
        for y in range(height):
            # Apply the transformation to the coordinates of the current pixel
            transformed_point = np.dot(affine_matrix, [x, y, 1])
            x_prime, y_prime = transformed_point[:2] / transformed_point[2]

            # Perform bilinear interpolation to get the color of the transformed pixel
            if 0 <= x_prime < panorama_width and 0 <= y_prime < panorama_height:
                x0, y0 = int(x_prime), int(y_prime)
                x1, y1 = x0 + 1, y0 + 1
                if x1 < panorama_width and y1 < panorama_height:
                    c00 = img1[y, x]
                    c01 = img2[int(y_prime), int(x_prime)]
                    c10 = img1[y, x]
                    c11 = img2[int(y_prime), int(x_prime)+1]
                    tx, ty = x_prime - x0, y_prime - y0
                    panorama[y0, x0] = (1-tx)*(1-ty)*c00 + tx*(1-ty)*c10 + (1-tx)*ty*c01 + tx*ty*c11

    return panorama

In [ ]:
final_image1 = np.zeros((rows, cols, 3))
final_image1.shape
for i in range(r):
    for j in range(c):
        final_image1[i][j] = image[0][i][j]
        
for k in range(len(image)-1):
    for i in range(rows):
        for j in range(cols):
            shape = (3,1)
            point = np.zeros(shape)
            new_point = np.zeros(shape)
            point[0][0] = j
            point[1][0] = i
            point[2][0] = 1
            new_point = np.matmul(affine_cumm[k], point)
            x_prime = new_point[0][0]
            y_prime = new_point[1][0]
            if(x_prime<0 or y_prime<0 or x_prime >= c or y_prime >= r ):
                continue
                
            x_ceil = math.ceil(x_prime)
            if(x_ceil >= c ):
                x_ceil = c - 1

            y_ceil = math.ceil(y_prime)
            if(y_ceil >= r):
                y_ceil = r  -1 

            x_floor = math.floor(x_prime)
            if(x_floor >= c ):
                x_floor = c - 1

            y_floor = math.floor(y_prime)
            if(y_floor >= r):
                y_floor = r-1 
            
            sum_intensity = np.zeros(3)
            int1 = abs(y_floor - i) * abs(x_floor - j)
            int2 = abs(y_floor - i) * abs(x_ceil - j)
            int3 = abs(y_ceil - i) * abs(x_floor - j)
            int4 = abs(y_ceil - i) * abs(x_ceil - j)
            totalIntensity = int1 + int2 + int3 + int4
            if (totalIntensity == 0):
                final_image1[i][j] = image[k+1][y_floor][x_floor]
            else :
                sum_intensity = (int1/totalIntensity) * image[k+1][y_floor][x_floor] + (int2/totalIntensity) * image[k+1][y_floor][x_ceil]  + (int3/totalIntensity) * image[k+1][y_ceil][x_floor]  + (int4/totalIntensity) * image[k+1][y_ceil][x_ceil]
                final_image1[i][j] = sum_intensity

In [ ]:
cv.imwrite('D:\\Mtech 2022-24\\Semester 2\\Computer Vision Assignment\\Assignment2\\New Dataset\\pano\\Panoroma_2check.jpg',final_image1)
cv.waitKey(0)
cv.destroyAllWindows()

## End